<a href="https://colab.research.google.com/github/ayyucedemirbas/Emotion_Detection/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/fer2013-20211101T153413Z-001.zip

In [1]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D,MaxPooling2D
import os

In [2]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
import PIL

In [4]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [5]:
num_classes=6
img_rows,img_cols=48,48
batch_size=64

In [6]:
train_data_dir='fer2013/train'
validation_data_dir='fer2013/validation'

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        color_mode='grayscale',
                        target_size=(img_rows,img_cols),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                validation_data_dir,
                                color_mode='grayscale',
                                target_size=(img_rows,img_cols),
                                batch_size=batch_size,
                                class_mode='categorical',
                                shuffle=True)

Found 28353 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


In [9]:
model = Sequential([
              Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
              input_shape=(img_rows,img_cols,1)),
              Activation('elu'),
              BatchNormalization(),
              Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                     input_shape=(img_rows,img_cols,1)),
              Activation('elu'),
              BatchNormalization(),
              MaxPooling2D(pool_size=(2,2)),
              Dropout(0.2),



            Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'),
            Activation('elu'),
            BatchNormalization(),
            Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'),
            Activation('elu'),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2,2)),
            Dropout(0.2),


          Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'),
          Activation('elu'),
          BatchNormalization(),
          Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'),
          Activation('elu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2)),
          Dropout(0.2),



        Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'),
        Activation('elu'),
        BatchNormalization(),
        Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'),
        Activation('elu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),
        Dropout(0.2),



        Flatten(),
        Dense(64,kernel_initializer='he_normal'),
        Activation('elu'),
        BatchNormalization(),
        Dropout(0.5),



        Dense(64,kernel_initializer='he_normal'),
        Activation('elu'),
        BatchNormalization(),
        Dropout(0.5),


        Dense(num_classes,kernel_initializer='he_normal'),
        Activation('softmax')
])

In [10]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [checkpoint,reduce_lr]

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=50

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [12]:
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/50
377/377 [==============================] - ETA: 0s - loss: 2.1569 - accuracy: 0.1958
Epoch 00001: val_loss improved from inf to 1.76665, saving model to EmotionDetectionModel.h5
377/377 [==============================] - 37s 88ms/step - loss: 2.1569 - accuracy: 0.1958 - val_loss: 1.7667 - val_accuracy: 0.2374 - lr: 0.0010
Epoch 2/50
377/377 [==============================] - ETA: 0s - loss: 1.8161 - accuracy: 0.2221
Epoch 00002: val_loss improved from 1.76665 to 1.74663, saving model to EmotionDetectionModel.h5
377/377 [==============================] - 32s 85ms/step - loss: 1.8161 - accuracy: 0.2221 - val_loss: 1.7466 - val_accuracy: 0.2551 - lr: 0.0010
Epoch 3/50
377/377 [==============================] - ETA: 0s - loss: 1.7537 - accuracy: 0.2464
Epoch 00003: val_loss improved from 1.74663 to 1.73160, saving model to EmotionDetectionModel.h5
377/377 [==============================] - 33s 88ms/step - loss: 1.7537 - accuracy: 0.2464 - val_loss: 1.7316 - val_accuracy: 0.2588 